# Sudoku recognition & solver

## Adapt dataset

sudoku images & dat files found on kaggle ([source](https://www.kaggle.com/datasets/mexwell/sudoku-image-dataset))

we have dataset but with an arborescence like this :
```sh
├── mixed
│   └── mixed
├── mixed 2
│   └── mixed 2
├── v1_test
│   └── v1_test
├── v1_training
│   └── v1_training
├── v2_test
│   └── v2_test
└── v2_train
    └── v2_train
```

With YOLOv8, we need to have a dataset like this :

([https://www.kaggle.com/datasets/meeratif/yolo-format-data](https://www.kaggle.com/datasets/meeratif/yolo-format-data))

```sh
dataset
├── images
│   ├── train
│   │   ├── sudoku_1.jpg
│   │   └── sudoku_2.jpg
│   └── val
│       ├── sudoku_3.jpg
│       └── sudoku_4.jpg
└── labels
    ├── train
    │   ├── sudoku_1.txt
    │   └── sudoku_2.txt
    └── val
        ├── sudoku_3.txt
        └── sudoku_4.txt
```

on dévellope un script pour adapter les fichiers d'annotations en quelque chose de viable pour YOLOv8 :

```sh
<class_id> <x_center> <y_center> <width> <height>
```

In [1]:
import os

In [2]:
def dat_to_yolo(dat_path, output_txt_path):
    """
    Convertit un fichier .dat de Sudoku en un fichier .txt au format YOLO.
    Args:
        dat_path (str): Chemin vers le fichier .dat.
        output_txt_path (str): Chemin de sortie pour le fichier YOLO .txt.
    """
    with open(dat_path, 'r') as file:
        lines = file.readlines()

    # Extraire les dimensions de l'image
    width, height = 0, 0
    for line in lines:
        if 'JPG' in line:
            resolution = line.split()[0]  # Extrait "640x480:24"
            resolution = resolution.split(':')[0]  # Supprime ":24"
            width, height = map(int, resolution.split('x'))
            break

    # Extraire la grille
    grid = []
    for line in lines:
        # Vérifiez si la ligne contient uniquement des chiffres (grille Sudoku)
        if line.strip() and all(char.isdigit() or char.isspace() for char in line):
            grid.append([int(x) for x in line.split()])

    # Dimensions des cellules dans la grille
    cell_width = 1 / 9  # Chaque cellule est 1/9 de la largeur
    cell_height = 1 / 9  # Chaque cellule est 1/9 de la hauteur

    # Conversion de la grille en annotations YOLO
    with open(output_txt_path, 'w') as output_file:
        for row_idx, row in enumerate(grid):
            for col_idx, value in enumerate(row):
                if value == 0:  # Ignorer les cases vides
                    continue
                x_center = (col_idx + 0.5) * cell_width
                y_center = (row_idx + 0.5) * cell_height
                width = cell_width
                height = cell_height
                class_index = value - 1  # Classes YOLO : 0 pour "1", 1 pour "2", etc.
                output_file.write(f"{class_index} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


# Antoinnnneee pas besoin de re run ça jlai déjà fait check le /dataset

In [ ]:
train_source = 'archive/v1_training/v1_training/'

files = os.listdir(train_source)
for file in files:
    if file.endswith('.dat'):
        dat_path = os.path.join(train_source, file)
        output_txt_path = os.path.join("dataset/labels/train/", file.replace('.dat', '.txt'))
        dat_to_yolo(dat_path, output_txt_path)
    if file.endswith('.jpg'):
        os.system(f"cp {train_source}{file} dataset/images/train/")
        
test_source = 'archive/v1_test/v1_test/'

files = os.listdir(test_source)
for file in files:
    if file.endswith('.dat'):
        dat_path = os.path.join(test_source, file)
        output_txt_path = os.path.join("dataset/labels/val/", file.replace('.dat', '.txt'))
        dat_to_yolo(dat_path, output_txt_path)
    if file.endswith('.jpg'):
        os.system(f"cp {test_source}{file} dataset/images/val/")

redimensionner les images en 640x640


In [2]:
import os
from PIL import Image

# Chemin vers les répertoires des images
image_dirs = ["datasets/images/train", "datasets/images/val"]
target_size = (640, 640)

# Redimensionnement des images
for image_dir in image_dirs:
    for image_name in os.listdir(image_dir):
        if image_name.endswith(('.jpg', '.png')):
            image_path = os.path.join(image_dir, image_name)
            with Image.open(image_path) as img:
                img_resized = img.resize(target_size)
                img_resized.save(image_path)

### Training sudoku recognition model

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from sudoku import affiche
import os

In [5]:
model = YOLO("yolov8n.pt")
model.train(data="sudoku_config.yaml", epochs=25, batch=16, imgsz=640)

Ultralytics 8.3.48 🚀 Python-3.12.3 torch-2.5.1+cu124 CPU (13th Gen Intel Core(TM) i9-13900H)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=sudoku_config.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train16, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

train: Scanning /home/erwan/Documents/TPS/2A/sudoku_python/datasets/labels/train.cache... 120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<?, ?it/s]
val: Scanning /home/erwan/Documents/TPS/2A/sudoku_python/datasets/labels/val.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]

Plotting labels to runs/detect/train16/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train16
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/25         0G      3.145      4.579      2.728        354        640: 100%|██████████| 8/8 [00:21<00:00,  2.71s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.58s/it]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:00<?, ?it/s]

WARNING ⚠️ NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:07<00:00,  3.56s/it]


                   all         40       1156     0.0353      0.112     0.0229     0.0135
                     1         40        131      0.043      0.084     0.0178      0.012
                     2         40        119     0.0264     0.0588     0.0138      0.007
                     3         40        129     0.0278      0.116     0.0236     0.0135
                     4         40        122      0.045      0.123     0.0355     0.0245
                     5         40        130     0.0419     0.0615     0.0269     0.0154
                     6         40        111     0.0214      0.207     0.0179    0.00919
                     7         40        133     0.0253      0.015     0.0116    0.00862
                     8         40        140     0.0334      0.279     0.0392     0.0203
                     9         40        141     0.0537     0.0638     0.0202     0.0109
Speed: 0.6ms preprocess, 57.7ms inference, 0.0ms loss, 116.9ms postprocess per image
Results saved to runs/det

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f76012e2480>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [ ]:
model_path = "models/yolov8n_trained_25.pt"
model.save(model_path)

In [ ]:
model = YOLO(model_path)
model.train(data="sudoku_config.yaml", epochs=25, batch=16, imgsz=640)

model_path = "models/yolov8n_trained_50.pt"
model.save(model_path)

Ultralytics 8.3.48 🚀 Python-3.12.3 torch-2.5.1+cu124 CPU (13th Gen Intel Core(TM) i9-13900H)
engine/trainer: task=detect, mode=train, model=models/yolov8n_trained_25.pt, data=sudoku_config.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train17, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

train: Scanning /home/erwan/Documents/TPS/2A/sudoku_python/datasets/labels/train.cache... 120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<?, ?it/s]
val: Scanning /home/erwan/Documents/TPS/2A/sudoku_python/datasets/labels/val.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]

Plotting labels to runs/detect/train17/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train17
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/25         0G      1.954      3.772      1.566        354        640: 100%|██████████| 8/8 [00:21<00:00,  2.67s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.67s/it]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/25         0G      1.759      3.622      1.449        367        640: 100%|██████████| 8/8 [00:22<00:00,  2.85s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.50s/it]

                   all         40       1156     0.0254      0.257     0.0239     0.0133



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/8 [00:00<?, ?it/s]